In [1]:
import newspaper
import sys
import json
import os
sys.path.append("../src/")
from build_llama_index import ARTICLES_JSONS_PATH
from newspaper_utils import get_article
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'
config = newspaper.Config()
config.browser_user_agent = user_agent
config.request_timeout = 10

In [7]:
def download_articles_from_list(url_list):
    num_previous_articles = 59
    good_articles = []
    for i, url in enumerate(url_list):
        article = get_article(url, verbose=True)

        # filter out bad articles
        if article["text"] != "" or \
            article["text"] != " " or \
            (url.contains("cnn.com") and article["text"].endswith("Read More")) or \
            url.contains("bbc.com"):
            date = article["date"].strftime("%Y-%m-%d %H:%M:%S") if article["date"] else None
            good_articles.append({"article_id":num_previous_articles+i,
            "url": url, "title": article["title"], "text": article["text"], "date": date})

    # save good articles to jsons
    article_file_name = "article_extra_{article_id}.json"
    for article in good_articles:
        with open(os.path.join(ARTICLES_JSONS_PATH, article_file_name.format(article_id=article["article_id"])), "w") as f:
            json.dump(article, f, indent=4)

In [4]:
websites = ["reuters.com/technology","techcrunch.com"]
article_urls = []
for website in websites:
    print(f"Getting articles from {website}")
    paper = newspaper.build(f"https://{website}", config=config, language='en', memoize_articles=False)
    print([article.url for article in paper.articles])
    for article in paper.articles:
        if website in article.url:
            article_urls.append(article.url)
            print(article.url)
            article = get_article(article.url, verbose=True)
            print(article["text"])

Getting articles from reuters.com/technology
['https://reuters.com/world/us/us-weekly-jobless-claims-unexpectedly-fall-2023-07-20/', 'https://reuters.com/business/retail-consumer/worlds-cheapest-dominos-pizza-is-inflation-hit-india-it-costs-060-2023-07-19/', 'https://reuters.com/markets/commodities/india-prohibits-export-non-basmati-white-rice-notice-2023-07-20/', 'https://reuters.com/technology/n-korea-hackers-breached-us-it-company-bid-steal-crypto-sources-2023-07-20/', 'https://reuters.com/markets/commodities/cell-cultivated-meat-hits-menus-investors-see-scaling-next-hurdle-2023-07-20/', 'https://reuters.com/business/autos-transportation/made-russia-chinese-cars-drive-revival-russias-auto-factories-2023-07-20/', 'https://reuters.com/world/swedish-embassy-baghdad-stormed-set-alight-source-witness-2023-07-19/', 'https://reuters.com/world/middle-east/who-is-iraqi-shiite-cleric-muqtada-al-sadr-2023-07-20/', 'https://reuters.com/business/environment/us-china-climate-talks-brought-goodwil

In [8]:
download_articles_from_list(article_urls)